<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# ── Install folium for interactive map visualizations ──
!pip3 install --quiet folium

# ── Install wget to download files programmatically ──
!pip3 install --quiet wget

In [2]:
# ── Import pandas for data manipulation ──
import pandas as pd

# ── Import folium for interactive maps ──
import folium

# ── Import wget for downloading external files ──
import wget

# ── Import MarkerCluster plugin for grouping map markers ──
from folium.plugins import MarkerCluster

# ── Import MousePosition plugin to show coordinates on hover ──
from folium.plugins import MousePosition

# ── Import DivIcon to create custom HTML labels on the map ──
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# ── Download the SpaceX launch geo dataset ──
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

# ── Read the CSV file into a pandas DataFrame ──
spacex_df = pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [4]:
# ── Select relevant columns: Launch Site, Latitude, Longitude, and Class ──
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# ── Group by Launch Site and keep the first occurrence of each ──
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# ── Keep only the Launch Site name and its coordinates ──
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# ── Display the resulting DataFrame with unique launch site coordinates ──
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# ── Define start location as NASA Johnson Space Center coordinates ──
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# ── Create a folium map centered at NASA coordinates with initial zoom ──
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# ── Create a colored circle at NASA Johnson Space Center with a popup label ──
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# ── Create a marker with a custom text label using DivIcon ──
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
)

# ── Add the circle and marker to the map ──
site_map.add_child(circle)
site_map.add_child(marker)

# ── Display map directly in notebook ──
site_map


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [7]:
# ── Initialize the folium map centered at NASA coordinate ──
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# ── Loop through each launch site and add markers and popup labels ──
for index, row in launch_sites_df.iterrows():
    name = row['Launch Site']
    latitude = row['Lat']
    longitude = row['Long']
    coordinate = [latitude, longitude]
    
    # ── Add a circle marker with popup label ──
    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(name))
    
    # ── Add a custom text marker with launch site name ──
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html="<div style='font-size: 12px; color:#d35400;'><b>{}</b></div>".format(name)
        )
    )

    # ── Add both elements to the map ──
    site_map.add_child(circle)
    site_map.add_child(marker)

In [8]:
# ── Render the folium map with launch site markers ──
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
# ── Display the last 10 rows of the SpaceX launch dataset ──
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
# ── Initialize a MarkerCluster object to group nearby markers ──
marker_cluster = MarkerCluster()

### Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# ── Create a new column 'marker_color' based on the launch outcome (class) ──
# Success (class = 1) → green, Failure (class = 0) → red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [12]:
# ── Define function to assign color based on launch outcome ──
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# ── Apply the function to create 'marker_color' column in the dataset ──
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# ── Display the last 10 rows to verify the new column ──
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# ── Add the MarkerCluster object to the folium map ──
site_map.add_child(marker_cluster)

# ── Loop through each launch record and add a colored marker based on outcome ──
for index, record in spacex_df.iterrows():
    name = record['Launch Site']
    latitude = record['Lat']
    longitude = record['Long']
    coordinate = [latitude, longitude]
    color = record['marker_color']

    # ── Create a marker with popup label and outcome-based color ──
    marker = folium.map.Marker(
        location=coordinate,
        popup=name,
        icon=folium.Icon(color='white', icon_color=color)
    )

    # ── Add the marker to the MarkerCluster ──
    marker_cluster.add_child(marker)

# ── Render the final map with marker clusters ──
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# ── Define JavaScript formatter to control coordinate precision ──
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# ── Initialize the MousePosition plugin to show coordinates on hover ──
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# ── Add the MousePosition plugin to the map ──
site_map.add_child(mouse_position)

# ── Render the map with mouse position enabled ──
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
# ── Import required trigonometric and conversion functions ──
from math import sin, cos, sqrt, atan2, radians

# ── Define function to calculate geodesic distance using Haversine formula ──
def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of Earth in kilometers
    R = 6373.0

    # Convert coordinates from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Compute differences in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Apply the Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate and return distance
    distance = R * c
    return distance


Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# ── Filter the dataset for launches from the VAFB SLC-4E site ──
spacex_df[spacex_df['Launch Site'] == 'VAFB SLC-4E']

,Launch Site,Lat,Long,class,marker_color
26,VAFB SLC-4E,34.632834,-120.610745,0,red
27,VAFB SLC-4E,34.632834,-120.610745,0,red
28,VAFB SLC-4E,34.632834,-120.610745,1,green
29,VAFB SLC-4E,34.632834,-120.610745,1,green
30,VAFB SLC-4E,34.632834,-120.610745,1,green
31,VAFB SLC-4E,34.632834,-120.610745,1,green
32,VAFB SLC-4E,34.632834,-120.610745,0,red
33,VAFB SLC-4E,34.632834,-120.610745,0,red
34,VAFB SLC-4E,34.632834,-120.610745,0,red
35,VAFB SLC-4E,34.632834,-120.610745,0,red


In [17]:
# ── Define coordinates for VAFB SLC-4E launch site ──
launch_site_lat = 34.632834
launch_site_lon = -120.610746

# ── Define coordinates for the nearest coastline point ──
coastline_lat = 34.63543
coastline_lon = -120.62508

# ── Calculate the geodesic distance to the coastline ──
distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# ── Define coordinates of the closest coastline point ──
coordinate = [34.63543, -120.62508]

# ── Create a folium marker to display the distance from the launch site ──
distance_marker = folium.map.Marker(
    location=coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)

# ── Add the distance marker to the map ──
site_map.add_child(distance_marker)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# ── Import all plugins from the folium.plugins module ──
import folium.plugins as plugins

In [20]:
# ── Define a function to draw a line from the launch site to a specified location ──
def draw_line(launch_site_lat, launch_site_lon, place_lat, place_lon, place_name):
    # ── Define coordinates for launch site and target place ──
    launch_coordinate = [launch_site_lat, launch_site_lon]
    place_coordinate = [place_lat, place_lon]

    # ── Add a red marker at the destination location ──
    marker = folium.map.Marker(
        location=place_coordinate,
        popup=place_name,
        tooltip='click',
        icon=folium.Icon(color='red')
    )
    site_map.add_child(marker)

    # ── Build coordinate list for polyline ──
    coordinates = [launch_coordinate, place_coordinate]

    # ── Calculate distance between the two points ──
    distance = calculate_distance(launch_site_lat, launch_site_lon, place_lat, place_lon)

    # ── Add a distance label using BeautifyIcon plugin ──
    text = "{} {:0.2f} km".format(place_name, distance)
    distance_marker = folium.map.Marker(
        location=place_coordinate,
        tooltip=text,
        icon=plugins.BeautifyIcon(
            number=round(distance, 0),
            border_color='#8D208B',
            border_width=2,
            text_color='#8D208B',
            inner_icon_style='margin-top:0px;'
        )
    )
    site_map.add_child(distance_marker)

    # ── Draw a polyline between the launch site and the target location ──
    linetext = "{} {:0.2f} km".format(place_name, distance)
    folium.PolyLine(coordinates, color='#325931', weight=1.5, popup=linetext).add_to(site_map)

In [21]:
# ── Define coordinates for the VAFB SLC-4E launch site ──
launch_site_lat = 34.632834
launch_site_lon = -120.610746

# ── Define coordinates for the nearest coastline ──
coastline_lat = 34.63543
coastline_lon = -120.62508

# ── Draw a line and marker showing the distance to the coastline ──
draw_line(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon, "Coastline")

# ── Display the updated map with polyline and distance marker ──
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [22]:
# ── Draw a line and marker from the launch site to the nearest city ──
draw_line(launch_site_lat, launch_site_lon, 34.63858, -120.45839, "Lompoc city")

# ── Draw a line and marker from the launch site to the closest railway ──
draw_line(launch_site_lat, launch_site_lon, 34.63263, -120.62507, "Santa Barbara Subdivision MT1")

# ── Draw a line and marker from the launch site to the nearest highway ──
draw_line(launch_site_lat, launch_site_lon, 34.63469, -120.61575, "Agena Way")

# ── Render the final map with all distance markers and polylines ──
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
